<a href="https://colab.research.google.com/github/AbdulrahmanAlmalkii/ArCyb/blob/main/ArCyb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/aub-mind/arabert/
!pip install tnkeeh
!pip install gensim
!pip install  farasapy
!pip install pyarabic
from arabert.preprocess import ArabertPreprocessor
from snowballstemmer import stemmer
from nltk.featstruct import remove_variables
from nltk.tokenize.casual import remove_handles
import tnkeeh as tn 
from datasets import load_dataset

from arabert.preprocess import ArabertPreprocessor
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
import nltk
from nltk.tokenize import word_tokenize
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:


stop_words = pd.read_csv(r'./stopwords.csv')

def preprocess(text):
    '''
    text is an arabic string input

    the preprocessed text is returned
    '''

    # remove punctuations
    # remove Tashkeel
    # remove longation
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ا", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    text = re.sub("n", " ", text)
    text = re.sub("انو", "انه", text)
    text = re.sub("دا", "هذا", text)
    text = re.sub("ذا", "هذا", text)
    text = re.sub("ليه", "لماذا", text)
    text = re.sub("ليش", "لماذا", text)
    text = re.sub("صلي", "صلا", text)
    text = re.sub("صلى", "صلا", text)
    text = re.sub(r'\((.*?)\)', '', text)
    text = re.sub('[A-Za-z]+', '', text)
    text = re.sub(r'\\u[A-Za-z0-9\\]+', '', text)

    text = ' '.join(word for word in text.split() if word not in stop_words and not word.isdigit())
    return text


def remove_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    return text


df = pd.read_excel(r"4140 final dataset.xlsx")
df = df.dropna()
df = df.sample(frac=1)
print(df.head())
print(df.sample(5))
print(df['label'].value_counts())

cleander = tn.Tnkeeh(remove_diacritics=True, remove_english=True, remove_twitter_meta=True, remove_tatweel=True,
                     remove_html_elements=True, remove_special_chars=True, remove_long_words=True,
                     remove_repeated_chars=True
                     , remove_links=True)
cleaned_dataset = cleander.clean_data_frame(df, 'Tweets')


def cleaning(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)

    text = re.sub("معوق", "معاق", text)
    text = re.sub("اعاقة", "معاق", text)
    text = re.sub("تبكي", "بكى", text)
    text = re.sub("يبكي", "بكى", text)
    text = re.sub("ببكي", "بكى", text)
    text = re.sub("بكا", "بكى", text)
    text = re.sub("شذوذ", "شذ", text)
    text = re.sub("شاذ", "شذ", text)
    text = re.sub("تصيح", "صاح", text)
    text = re.sub("بصيح", "صاح", text)
    text = re.sub("يصيح", "صاح", text)
    text = re.sub("صياح", "صاح", text)
    text = re.sub("يتدلع", "دلع", text)
    text = re.sub("تتدلع", "دلع", text)
    text = re.sub("احنا", "حنى", text)
    text = re.sub("منا", "من", text)
    text = re.sub("كانو", "كانه", text)
    text = re.sub("منو", "منه", text)
    text = re.sub("زيو", "زيه", text)
    text = re.sub("انو", "انه", text)
    text = re.sub("هذا العبد", "عبد", text)
    text = re.sub("يا العبد", "عبد", text)
    text = re.sub("العبد", "عبد", text)

    text = re.sub('[A-Za-z]+', ' ', text)
    text = re.sub(r'\\u[A-Za-z0-9\\]+', ' ', text)
    text = re.sub('[0-9]+', ' ', text)

    return text

cleaned_dataset['Tweets'] = cleaned_dataset['Tweets'].apply(cleaning)

feature = df.Tweets.astype(str)

target = df.label.astype(str)


model_name = "aubmindlab/bert-base-arabertv2"
arabert_prep = ArabertPreprocessor(model_name=model_name)
feature = np.array(cleaned_dataset['Tweets'])
for i in range(len(feature)):
    feature[i] = arabert_prep.preprocess(feature[i])
    feature[i] = preprocess(feature[i])
    feature[i] = remove_emoji(feature[i])


def remove_stop_words(lines):
    stop_words = ['+']
    results = []
    for text in lines:
        tmp = text.split(' ')
        for x in range(0, len(tmp)):
            for st_w in stop_words:
                if st_w in tmp[x]:
                    tmp[x] = ''
        results.append(" ".join(tmp))
    return results



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import SGDClassifier

X_train, X_test, Y_train, Y_test = train_test_split(feature, target, test_size=.2, stratify=target )


pipe = make_pipeline(CountVectorizer(),
                    TfidfTransformer(),
                     MLPClassifier(solver='adam', hidden_layer_sizes=(30,66,66, 30),
                                   activation='logistic', learning_rate='adaptive', shuffle=True,  validation_fraction=0.01, max_iter=600)
                     )

mlp_model = pipe.fit(X_train,Y_train)

prediction = mlp_model.predict(X_test)
print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")
print(classification_report(Y_test, prediction))
print(confusion_matrix(prediction, Y_test))

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

tf_vec = TfidfVectorizer()
X = tf_vec.fit_transform(feature)


X_train, X_test, Y_train, Y_test = train_test_split(X, target, test_size=.2, random_state=16,  stratify=target )

svc = SVC()
svc.fit(X_train, Y_train)
preds = svc.predict(X_test)
acc = np.mean(preds == Y_test)
print('SVC model accuracy: {}'.format(acc*100))

print(f"Accuracy score is {accuracy_score(Y_test, preds):.2f}")
print(classification_report(Y_test, preds))

print(confusion_matrix(preds, Y_test))
